### In this jupyter notebook, we will practice reading catalog in FITS file, make sample selections, and take a look at the galaxies by making cropped images centered at them. In this exmple, we use catalog of galaxies detected in the NGDEEP field

In [1]:
import os
import random
import numpy as np
import pandas as pd
import astropy.stats
from astropy.io import fits
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] ='Times New Roman'

## (1) Read the galaxy catalog and get some basic parameters (e.g., coordinates, redshift, mass, and SFR)

In [2]:
dir = 'D:/imaging_data/CEERS/ceers-full-grizli-v6.0-f115w-clear_drc_wht.fits.gz'

sci = fits.open(dir)
sci.info()
sci[0].header

sci_data = sci[0].header
sci_data

Filename: D:/imaging_data/CEERS/ceers-full-grizli-v6.0-f115w-clear_drc_wht.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU    1101   (36864, 12288)   float32   


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                36864                                                  
NAXIS2  =                12288                                                  
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =              18432.0 / Pixel coordinate of reference point            
CRPIX2  =               6144.0 / Pixel coordinate of reference point            
CD1_1   = -7.1420845520725E-06 / Coordinate transformation matrix element       
CD1_2   = -8.5116049235440E-06 / Coordinate transformation matrix element       
CD2_1   = -8.5116049235440E-06 / Coordinate transformation matrix element       
CD2_2   = 7.14208455207265E-06 / Coordinate transformation matrix element       
CDELT1  =                  1

In [3]:
photmjsr = sci_data['PHOTMJSR']
photfnu = sci_data['PHOTFNU']
pixar = sci_data['PIXAR_SR']
#calib = photmjsr*pixar*10**12
calib = photfnu*10**6

sci.close()

In [4]:
# Directory of the catalog: where the catalog is located
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/All Catalogs/'

cat_eazy = fits.open(cat_dir+'allcats_zout.fits')
cat_eazy.info()
cat_eazy[1].header

Filename: D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/All Catalogs/allcats_zout.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (19767,)   uint8   
  1  /Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca    1 BinTableHDU    199   216098R x 83C   [K, 14A, D, D, D, K, E, E, E, D, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, 5D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E]   


XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                  978 / width of table in bytes                        
NAXIS2  =               216098 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                   83 / number of columns                              
EXTNAME = '/Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca'
TTYPE1  = 'index   '           / label for column 1                             
TFORM1  = 'K       '           / format for column 1                            
TCOMM1  = 'Expression: index'                                                   
TTYPE2  = 'ID_new  '        

In [5]:
# get the catalog data
cat_eazy_data = cat_eazy[1].data
cat_eazy_data

FITS_rec([(     1, 'CEERS1', 215.21529079, 52.97537283, -1., 0, -1.        , -1.        , -1.        , 4.31882810e+13,     0.        , -1.        , -1.        , -1.        , 0.01      , 0.        , 0.01      ,  0.       , 0.        , 0.        , 0.        , 0.       , 0.       , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ,     0.        , -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29], [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29], [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29], [-9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.00000000e+29, -9.000

In [6]:
# Combined ID from all fields
ID_new = cat_eazy_data['ID_new']

# Right ascension and declination
ra = cat_eazy_data['ra']
dec = cat_eazy_data['dec']

# redshifts: spectroscopic and photometric redshifts
z_spec = cat_eazy_data['z_spec']
z_phot = cat_eazy_data['z_phot']

# rest-frame magnitudes in U, V, and J bands
restU = cat_eazy_data['restU']
restV = cat_eazy_data['restV']
restJ = cat_eazy_data['restJ']

# luminosity in U, V, and J bands
Lu = cat_eazy_data['Lu']
Lv = cat_eazy_data['Lv']
Lj = cat_eazy_data['Lj']

# stellar mass (SM) and star formation rate (SFR)
SM = cat_eazy_data['mass']
SFR = cat_eazy_data['sfr']
SM_p = cat_eazy_data['mass_p']
SM1 = np.array([x[1] for x in SM_p])
SM3 = np.array([x[3] for x in SM_p])
SFR_p = cat_eazy_data['sfr_p']
SFR1 = np.array([x[1] for x in SFR_p])
SFR3 = np.array([x[3] for x in SFR_p])

# number of objects (i.e., galaxies)
ngals = len(ra)
print ('Number of galaxies in the catalog: %d' % ngals)

cat_eazy.close()

Number of galaxies in the catalog: 216098


## (2) Read the photometry catalog

In [7]:
cat_photo = fits.open(cat_dir+'allcats_phot_apcorr.fits')
cat_photo.info()
cat_photo[1].header

Filename: D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/All Catalogs/allcats_phot_apcorr.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (95519,)   uint8   
  1  /Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca    1 BinTableHDU   2554   216098R x 890C   [K, 14A, D, K, K, K, K, K, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, K, K, K, K, D, D, J, D, D, D, D, D, D, K, D, D, D, D, D, D, K, D, I, D, D, D, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D, D, I, D, D, D, D, D, I, D, D, D, D, I, D, D, D

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                 6576 / width of table in bytes                        
NAXIS2  =               216098 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                  890 / number of columns                              
EXTNAME = '/Users/itsnakhoirulfitriana/Documents/student_project_itb_galclum/ca'
TTYPE1  = 'index   '           / label for column 1                             
TFORM1  = 'K       '           / format for column 1                            
TCOMM1  = 'Expression: index'                                                   
TTYPE2  = 'ID_new  '        

In [8]:
# get the catalog data
cat_photo_data = cat_photo[1].data

rkron = cat_photo_data['kron_radius']
flux_iso = cat_photo_data['flux_iso']
flux_radius = cat_photo_data['flux_radius']
flux_radius20 = cat_photo_data['flux_radius_20']
flux_radius90 = cat_photo_data['flux_radius_90']

In [9]:
# check the list of filters and their names from the header information printed above
filter_names = ['f090w', 'f105w', 'f110w', 'f115w', 'f125w', 'f140w', 'f150w', 'f160w',
                'f182m', 'f200w', 'f210m', 'f277w', 'f335m', 'f356w', 'f410m', 'f430m',
                'f435w', 'f444w', 'f460m', 'f475w', 'f480m', 'f606w', 'f606wu', 'f775w',
                'f814w', 'f814wu', 'f850lp', 'f850lpu']
nfilters = len(filter_names)
print ('Number of filters: %d' % nfilters)

# get central wavelength of the filters
# filter_cwave = np.zeros(nfilters)
# for bb in range(nfilters):
#     filter_cwave[bb] = cat_photo[1].header[filter_names[bb]+'_PLAM']

Number of filters: 28


In [ ]:
# get the fluxes in units of micro Jansky (uJy)
# The measurements were done with circular aperture with 0.7 arcsecond diameter
# Now the fluxes have been corrected for the flux loss due to the small aperture 
flux = {}
flux_err = {}
for bb in range(nfilters):
    flux[filter_names[bb]] = np.zeros(ngals)
    flux_err[filter_names[bb]] = np.zeros(ngals)
    
for ii in range(ngals):
    for bb in range(nfilters):
        flux[filter_names[bb]][ii] = cat_photo_data[filter_names[bb]+'_tot_2'][ii]
        flux_err[filter_names[bb]][ii] = cat_photo_data[filter_names[bb]+'_etot_2'][ii]

In [11]:
# calculate AB magnitudes
ABmag = {}
for bb in range(nfilters):
    ABmag[filter_names[bb]] = np.zeros(ngals)
    
for ii in range(ngals):
    for bb in range(nfilters):
        ABmag[filter_names[bb]][ii] = -2.5*np.log10(flux[filter_names[bb]][ii]*1e-6) + 8.90

cat_photo.close()

C:\Users\Ryo\AppData\Local\Temp\ipykernel_19428\2960262805.py:8: RuntimeWarning: invalid value encountered in log10
  ABmag[filter_names[bb]][ii] = -2.5*np.log10(flux[filter_names[bb]][ii]*1e-6) + 8.90


## Segmentation

In [12]:
idx_bin = np.where((z_spec>1) & (z_spec<3) & (np.log10(SM)>8.5) & (np.log10(SM)<11.5) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
data = {'ID' : [ID_new[ii] for ii in idx_bin[0]], 
        'RA' : [ra[ii] for ii in idx_bin[0]],
        'Dec' : [dec[ii] for ii in idx_bin[0]],
        'z_sp' : [z_spec[ii] for ii in idx_bin[0]],
        'log_SM' : [np.log10(SM[ii]) for ii in idx_bin[0]],
        'log_SFR' : [np.log10(SFR[ii]) for ii in idx_bin[0]],
        'log_sSFR' : [np.log10(SFR[ii])-np.log10(SM[ii]) for ii in idx_bin[0]]}

df_sel = pd.DataFrame(data, index=[ii for ii in idx_bin[0]])
df_z = df_sel.sort_values(by=['z_sp'])
df_z

plt.rcParams['font.family'] ='sans serif'

C:\Users\Ryo\AppData\Local\Temp\ipykernel_18236\1459431274.py:1: RuntimeWarning: divide by zero encountered in log10
  idx_bin = np.where((z_spec>1) & (z_spec<3) & (np.log10(SM)>8.5) & (np.log10(SM)<11.5) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
C:\Users\Ryo\AppData\Local\Temp\ipykernel_18236\1459431274.py:1: RuntimeWarning: invalid value encountered in log10
  idx_bin = np.where((z_spec>1) & (z_spec<3) & (np.log10(SM)>8.5) & (np.log10(SM)<11.5) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flux['f277w']>0) & (flux['f444w']>0))
C:\Users\Ryo\AppData\Local\Temp\ipykernel_18236\1459431274.py:1: RuntimeWarning: divide by zero encountered in divide
  idx_bin = np.where((z_spec>1) & (z_spec<3) & (np.log10(SM)>8.5) & (np.log10(SM)<11.5) & (abs(z_spec-z_phot)/(1+z_spec)<0.15) & (flux['f160w']>0) & (flux['f115w']>0) & (flux['f150w']>0) & (flu

### CEERS

#### F115W

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/CEERS/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'ceers-full-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'ceers-full-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'ceers-full-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'ceers-full-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'ceers-full-grizli-v6.0-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'ceers-full-grizli-v6.0-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'ceers-full-grizli-v6.0-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'ceers-full-grizli-v6.0-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

segment = {}
segment = 'ceers-full-grizli-v6.0-ir_seg.fits.gz'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])

seg_pixsizes = {}
seg_pixsizes[segment] = calc_pixsize(dir_images+segment)
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/CEERS/'

seg_fits = fits.open(cat_dir+'ceers-full-grizli-v6.0-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'CEERS' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

for ii in list[265:]:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
        
        if bb == 0:
            FWHM = 1.2
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii]) 
                        
                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])

            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
        
    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP13-CEERS-3.csv')

#### F150W

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/CEERS/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'ceers-full-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'ceers-full-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'ceers-full-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'ceers-full-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'ceers-full-grizli-v6.0-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'ceers-full-grizli-v6.0-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'ceers-full-grizli-v6.0-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'ceers-full-grizli-v6.0-f444w-clear_drc_wht.fits.gz'


telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/CEERS/'

seg_fits = fits.open(cat_dir+'ceers-full-grizli-v6.0-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'CEERS6801' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

fil = [1, 0, 2, 3]
for ii in list:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)[xx]     
        
        if bb == 0:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            FWHM = 1.6
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, nlevels=256, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii])   
                        
                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])
                        
            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
        
    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
#clumpy.to_csv('CLUMP36-CEERS.csv')

### COSWEB

#### F115W

In [20]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/COSMOS-WEB/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'cosweb-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'cosweb-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'cosweb-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'cosweb-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'cosweb-grizli-v6.0-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'cosweb-grizli-v6.0-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'cosweb-grizli-v6.0-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'cosweb-grizli-v6.0-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

{'f115w': 0.03999999999999973, 'f150w': 0.03999999999999973, 'f277w': 0.03999999999999973, 'f444w': 0.03999999999999973}


In [23]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/COSMOS-WEB/'

seg_fits = fits.open(cat_dir+'cosweb-grizli-v6.0-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'COSWEB' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

for ii in list:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
        
        if bb == 0:
            FWHM = 1.2
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii]) 
                        
                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])
                        
            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
        
    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP13-COSWEB.csv')

#### F150W

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/COSMOS-WEB/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'cosweb-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'cosweb-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'cosweb-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'cosweb-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'cosweb-grizli-v6.0-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'cosweb-grizli-v6.0-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'cosweb-grizli-v6.0-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'cosweb-grizli-v6.0-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/COSMOS-WEB/'

seg_fits = fits.open(cat_dir+'cosweb-grizli-v6.0-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'COSWEB' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

fil = [1, 0, 2, 3]
for ii in list:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)[xx]
        
        if bb == 0:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            FWHM = 1.6
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii]) 
                        
                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])
                        
            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
        
    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP36-COSWEB.csv')

### PRIMER-UDS

#### F115W

In [13]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/PRIMER-UDS/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'primer-uds-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'primer-uds-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'primer-uds-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'primer-uds-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'primer-uds-grizli-v6.0-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'primer-uds-grizli-v6.0-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'primer-uds-grizli-v6.0-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'primer-uds-grizli-v6.0-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

{'f115w': 0.03999999999999973, 'f150w': 0.03999999999999973, 'f277w': 0.03999999999999973, 'f444w': 0.03999999999999973}


In [15]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/PRIMER-UDS/'

seg_fits = fits.open(cat_dir+'sub-primer-uds-grizli-v6.0-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'PRIME' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

for ii in list[158:]:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)     
        
        if bb == 0:
            FWHM = 1.2
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii])
                        
                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])
                        
            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
        
    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP13-PRIMER-2.csv')

#### F150W

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/PRIMER-UDS/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'primer-uds-grizli-v6.0-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'primer-uds-grizli-v6.0-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'primer-uds-grizli-v6.0-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'primer-uds-grizli-v6.0-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'primer-uds-grizli-v6.0-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'primer-uds-grizli-v6.0-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'primer-uds-grizli-v6.0-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'primer-uds-grizli-v6.0-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/PRIMER-UDS/'

seg_fits = fits.open(cat_dir+'sub-primer-uds-grizli-v6.0-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'PRIME' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

fil = [1, 0, 2, 3]
for ii in list:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
        
        if bb == 0:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            FWHM = 1.6
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii])

                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])
                        
            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
        
    cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.3)

    d_A = cosmo.angular_diameter_distance(z=z_spec[ii])
    scalebar_l = 5 * u.kpc
    scalebar_t = (scalebar_l / d_A).to(u.deg, equivalencies=u.dimensionless_angles())

    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP36-PRIMER.csv')

### FRESCO

#### F115W

In [13]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/FRESCO/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'gds-grizli-v6.1-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'gds-grizli-v6.1-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'gds-grizli-v6.1-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'gds-grizli-v6.1-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'gds-grizli-v6.1-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'gds-grizli-v6.1-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'gds-grizli-v6.1-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'gds-grizli-v6.1-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

{'f115w': 0.019999999999999896, 'f150w': 0.019999999999999896, 'f277w': 0.03999999999999973, 'f444w': 0.03999999999999973}


In [19]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/FRESCO/'

seg_fits = fits.open(cat_dir+'gds-grizli-v6.1-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'FRESCO' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []

for ii in list[445:]:

    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
        
        if bb == 0:
            FWHM = 1.2
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii]) 

                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])

            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                if bb == 0 or bb == 1:
                    ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
                elif bb == 2 or bb == 3:
                    ax.add_patch(Circle((x0.item()-0.8, y0.item()-0.8), r_pix, lw=2.5, fill=False, color='cyan'))
                    
                    
    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP13-FRESCO-3.csv')

#### F150W

In [ ]:
import matplotlib.image as mpimg

# list of filters available in NGDEEP, taken from the information in the header
filter_names = ['f115w', 'f150w', 'f277w', 'f444w']

nfilters = len(filter_names)
# Directory where the imaging data are being stored
dir_images = 'D:/imaging_data/FRESCO/'

# make a python dictionary to store the list of images 
img_name = {} 
img_name['f115w'] = 'gds-grizli-v6.1-f115w-clear_drc_sci.fits.gz'
img_name['f150w'] = 'gds-grizli-v6.1-f150w-clear_drc_sci.fits.gz'
img_name['f277w'] = 'gds-grizli-v6.1-f277w-clear_drc_sci.fits.gz'
img_name['f444w'] = 'gds-grizli-v6.1-f444w-clear_drc_sci.fits.gz'

img_err = {}
img_err['f115w'] = 'gds-grizli-v6.1-f115w-clear_drc_wht.fits.gz'
img_err['f150w'] = 'gds-grizli-v6.1-f150w-clear_drc_wht.fits.gz'
img_err['f277w'] = 'gds-grizli-v6.1-f277w-clear_drc_wht.fits.gz'
img_err['f444w'] = 'gds-grizli-v6.1-f444w-clear_drc_wht.fits.gz'

telescope = {}
telescope['f115w'] = 'JWST'
telescope['f150w'] = 'JWST'
telescope['f277w'] = 'JWST'
telescope['f444w'] = 'JWST'

# get pixel scale
from piXedfit.piXedfit_images import calc_pixsize

img_pixsizes = {}
for bb in range(nfilters):
    img_pixsizes[filter_names[bb]] = calc_pixsize(dir_images+img_name[filter_names[bb]])
    
print (img_pixsizes)

In [ ]:
from astropy.wcs import WCS
from astropy.visualization import LogStretch
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization.wcsaxes import WCSAxes, add_scalebar
from astropy.nddata import Cutout2D
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.convolution import convolve, Gaussian2DKernel
from photutils.segmentation import deblend_sources, detect_sources, SourceCatalog, make_2dgaussian_kernel
from matplotlib.patches import Circle


%matplotlib inline

# Directory of segmentation
cat_dir = 'D:/Ryo/Documents/Academic/Kuliah/S1/Semester 7/AS4091 Tugas Akhir I/Catalogs/FRESCO/'

seg_fits = fits.open(cat_dir+'gds-grizli-v6.1-ir_seg.fits.gz', memmap=True)
grizz = np.array(seg_fits[0].data)

list = [df_z[df_z['ID'] == x].index.values[0] for x in df_z.ID if 'FRESCO' in x]

clumps_x = []
clumps_y = []
clumps_ra = []
clumps_dec = []
ID_idx = []
fil = [1, 0, 2, 3]
for ii in list[318:]:
    
    fig = plt.figure(figsize=(25, 12))

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    
    dim_arcsec = 2  #dimensi cutout dalam arcsec
    dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]

    stamp_size = [dim_pix,dim_pix]     
    
    wcs_seg = WCS(seg_fits[0].header)
    gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
    seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      

    seg_num = np.unique(seg_grizli.data)
    
    while len(seg_num) < 2:
        dim_arcsec += 1  #dimensi cutout dalam arcsec
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix]  
        
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)      
        seg_num = np.unique(seg_grizli.data)
        
    for bb in range(0,nfilters):
        
        dim_pix = dim_arcsec/img_pixsizes[filter_names[bb]]
        stamp_size = [dim_pix,dim_pix] 
            
        hdu_img = fits.open(dir_images+img_name[filter_names[bb]], memmap=True)
        hdu = hdu_img[0]
        wcs = WCS(hdu.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cutout = Cutout2D(hdu.data, position=(gal_x,gal_y), size=stamp_size, wcs=wcs)
        img_crop = cutout.data
        hdu_img.close()
        
        hdu_err = fits.open(dir_images+img_err[filter_names[bb]], memmap=True)
        err = hdu_err[0]
        wcserr = WCS(err.header)
        gal_x, gal_y = wcs.wcs_world2pix(ra[ii], dec[ii], 1)
        cuterr = Cutout2D(err.data, position=(gal_x, gal_y), size=stamp_size, wcs=wcserr)
        bkg = cuterr.data
        bkg = 1/np.sqrt(bkg)
        hdu_err.close()
        threshold = 2*bkg

        wcs_seg = WCS(seg_fits[0].header)
        gal_x, gal_y = wcs_seg.wcs_world2pix(ra[ii], dec[ii], 1)
        seg_grizli = Cutout2D(grizz, position=(gal_x,gal_y), size=stamp_size, wcs=wcs_seg)   
        
        index = [5, 6, 7, 8]
        ax = fig.add_subplot(2, 4, index[bb])
        norm = simple_norm(img_crop, 'asinh', percent=99.)
        im = ax.imshow(img_crop, norm=norm, origin='lower', cmap='Greys')
        ax.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
        
        if bb == 0:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
        elif bb == 1: 
            FWHM = 1.6
            size = np.floor(4*1.2+1)
            kernel = Gaussian2DKernel(0.5*1.2, x_size=size, y_size=size)
            conv_image = convolve(img_crop, kernel)
            ax1 = fig.add_subplot(2, 4, 1)
            ax1.imshow(conv_image, norm=norm, origin='lower', cmap='Greys')
            ax1.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            segment = detect_sources(conv_image, threshold, npixels=10)
            
            seg = fig.add_subplot(2, 4, 3)
            deb = fig.add_subplot(2, 4, 4)
            seg.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            deb.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            
            if segment is not None:
                seg.imshow(segment, origin='lower', cmap=segment.cmap, interpolation='nearest')
                segm_deblend = deblend_sources(conv_image, segment, npixels=2, contrast=0, progress_bar=False)
                deb.imshow(segm_deblend, origin='lower', cmap=segm_deblend.cmap, interpolation='nearest')

                cat = SourceCatalog(conv_image, segm_deblend, convolved_data=conv_image)
                tbl = cat.to_table()
                
                x = tbl['xcentroid']
                y = tbl['ycentroid']
                
                c_ra, c_dec = wcs.wcs_pix2world(x, y, 1)
                IDnumber = int(''.join(ID for ID in ID_new[ii] if ID.isdigit()))

                RA = []
                DEC = []
                for xx in range(len(x)):
                    x0, y0 = wcs.wcs_world2pix(c_ra[xx], c_dec[xx], 1)
                    if seg_grizli.data[round(y0.item())][round(x0.item())] == IDnumber:
                        clumps_x.append(x0.item())
                        clumps_y.append(y0.item())
                        clumps_ra.append(c_ra[xx])
                        clumps_dec.append(c_dec[xx])
                        ID_idx.append(ID_new[ii]) 
                        
                        RA.append(c_ra[xx])
                        DEC.append(c_dec[xx])

            elif segment is None:
                x = np.NaN
                y = np.NaN
                clumps_x.append(x)
                clumps_y.append(y)
                ID_idx.append(ID_new[ii])
                            
            plt.text(0.5, 0.05, 'SEGM', horizontalalignment='center', verticalalignment='center',
                transform = seg.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'SEGM DEBL', horizontalalignment='center', verticalalignment='center',
                transform = deb.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.05, 'CONV %s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

            plt.text(0.5, 0.9, '%s \nz = %lf' % (ID_new[ii],z_spec[ii]), horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='gold', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            
            cmap = mpl.cm.gray
            bounds = np.unique(seg_grizli.data)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
            
            ax1 = fig.add_subplot(2, 4, 2)
            ax1.imshow(seg_grizli.data, origin='lower', norm=norm, cmap=cmap)
            plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, bottom=False, left=False)

            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            plt.text(0.5, 0.05, 'SEGM GRIZLI', horizontalalignment='center', verticalalignment='center',
                transform = ax1.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 2:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])

        elif bb == 3:
            plt.text(0.5, 0.05, '%s %s' % (telescope[filter_names[bb]].upper(),filter_names[bb].upper()), horizontalalignment='center', verticalalignment='center',
                transform = ax.transAxes, fontsize=18, color='white', fontweight='bold', path_effects=[pe.withStroke(linewidth=5, foreground="black")])
            

        if segment is not None:
            for xx in range(len(RA)):
                r = 0.05
                r_pix = r/img_pixsizes[filter_names[bb]]
                x0, y0 = wcs.wcs_world2pix(RA[xx], DEC[xx], 1)
                if bb == 0 or bb == 1:
                    ax.add_patch(Circle((x0.item(), y0.item()), r_pix, lw=2.5, fill=False, color='cyan'))
                elif bb == 2 or bb == 3:
                    ax.add_patch(Circle((x0.item()-0.8, y0.item()-0.8), r_pix, lw=2.5, fill=False, color='cyan'))
                    
    name_out = 'segment_z_{:.4f}_{}.png'.format(z_spec[ii], ID_new[ii])
    plt.savefig(name_out, bbox_inches='tight')

clumpp = {'ID' : ID_idx, 
          'x' : clumps_x,
          'y' : clumps_y, 
          'A_clump' : Area_clump}

clumpy = pd.DataFrame(clumpp)
clumpy.to_csv('CLUMP36-FRESCO-3.csv')